In [1]:
# load needed python libraries matplotlib, numpy, and scikit-rf
import matplotlib.pyplot as plt
import numpy as np
import skrf as rf
import json
from scipy.optimize import curve_fit
data = rf.read_all('ampdata')
#fghz = data1['trace0'].f/1e9
fghz = data['trace0'].f/1e9
def lorentzian(x,x0,curvepeak,fwhm):
    return curvepeak/(1 + 4*((x-x0)/(fwhm))**2)


In [3]:
for index in range(100):
    graw = np.abs(data['trace' + str(index)].s[:,0,0])**2
    popt, pcov = curve_fit(lorentzian,fghz,graw,p0=(6.5,80,0.03))
    f0 = popt[0]
    fwhm = popt[1]
    curvepeak = popt[2]
    gaindb = 10*np.log10(graw)
    gaindbfit = 10*np.log10(lorentzian(fghz,f0,fwhm,curvepeak))
    tracedata = {}
    tracedata['fghz'] = fghz.tolist()
    tracedata['gaindb'] = gaindb.tolist()
    tracedata['gaindbfit'] = gaindbfit.tolist()
    tracedata['f0'] = popt[0]
    tracedata['curvepeak'] = 10*np.log10(popt[1])
    tracedata['fwhm'] = np.abs(popt[2])
    file_object = open("ampdatajson/trace" + str(index) + ".json", "w")
    json_string = file_object.write(json.dumps(tracedata))
    file_object.close()